# Introduction

Fix $n \in \mathbb{N}.$ Let $P \subset \mathbb{R}^n$ be a nonempty subset. Recall $P$ is a $\textbf{convex polytope}$ in $\mathbb{R}^n$ if it is the convex hull of $k$ distinct $\textbf{defining vertices}$ $\mathbf{v}_1, \ \dots \ , \mathbf{v}_k \in \mathbb{R}^n$ for some $k \ge n+1.$ More explicitly, $P$ is a polytope if there exist distinct $\mathbf{v}_1, \ \dots \ , \mathbf{v}_k \in \mathbb{R}^n$ with $k \ge n+1$ such that
$$P=\text{Conv}(\mathbf{v}_1, \ \dots \ , \mathbf{v}_k)= \left \lbrace \sum_{i=1}^k \lambda_i \mathbf{v}_i: \lambda_1, \ \dots \ , \lambda_k \ge 0, \sum_{i=1}^k \lambda_i =1 \right \rbrace.$$ 

In the special case $n=2,$ convex polytopes are the same as convex polygons. Examples include triangles, quadrilaterals, pentagons, hexagons, heptagons, octagons, etc. In the case $n=3,$ examples include tetrahedra, parallelepipeds, etc. 


The most common convex polytope in $\mathbb{R}^n$ is the simplex. A $\textbf{simplex}$ in $\mathbb{R}^n$ is a convex hull of $(n+1)$  $\textbf{affinely independent}$ vectors $\mathbf{w}_1, \ \dots \ , \mathbf{w}_{n+1} \in \mathbb{R}^n,$ which means the $\lbrace \mathbf{w}_{j+1} - \mathbf{w}_1 \rbrace_{j=1}^n$ is linearly independent in $\mathbb{R}^n$. In the case $n=1,$ a simplex is simply a line segment. In the case $n=2,$ a simplex is a triangle. In  In the case $n=3,$ the simplex is a tetrahedron. 


A classical problem in combinatorial topology with many practical applications (such as the Travelling Salesman Problem in Linear Programming) is to $\textbf{triangulate } P.$ This means we write $P$ into a union of simplices $S_1, \ \dots \ , S_{m}$ such that the $S_i, S_j$ have pairwise disjoint interiors for all distinct $i$ and $j.$ For example, triangulation of a convex polygon with $k$ vertices is fairly trivial (with $k >3$): one can fix any vertex and connect straight lines between the vertex in question and all non-adjacent vertices to that vertex. This decomposes the polygon into $k-3$ triangles whose interiors are pairwise disjoint. For all $n \in \mathbb{N},$ it is known the $n$-dimensional parallelepiped can be decomposed into $n!$ tetrahedra whose interiors are pairwise disjoint. 

In fact, a remarkable theorem is that any convex polytope $P$ can be decomposed into a union of simplices whose interiors are pairwise disjoint. In this notebook, we accomplish two things:

* We triangulate an arbitrary convex polytope $P$ given its defining vertices.
* We determine the volume of $P$ by summing up the volumes of simplices that make up the triangulation.

In the cell below, we import the necessary libraries. Please make sure the cell below runs.

In [19]:
import numpy as np
import scipy as sc
import scipy.special
import itertools
from joblib import Parallel, delayed

# Determining Candidate Simplices For Triangulation

Let $P$ be a convex polytope with defining vertices $V= \lbrace \mathbf{v}_1, \ \dots \ , \mathbf{v}_k \rbrace$ where $k \ge n+1.$ We determine simplices who are candidates for triangulation. In order to do this, we first obtain all subsets of $V$ with $n+1$ distinct elements. Each subset is of the form $$\lbrace \mathbf{v}_{i_1}, \ \dots \ , \mathbf{v}_{i_{n+1}} \rbrace,$$ where $1 \le i_1 < \ \dots \ < i_{n+1} \le k.$ Recall by elementary combinatorics, there are $\binom{k}{n+1}$ total subsets of this form. We determine if each aforementioned subset $\lbrace \mathbf{v}_{i_1}, \ \dots \ , \mathbf{v}_{i_{n+1}} \rbrace$ is affinely independent in $\mathbb{R}^n.$ That is, we determine whether $\lbrace \mathbf{v}_{i_{j+1}} - \mathbf{v}_{i_1} \rbrace_{j=1}^n$ is a linearly independent set in $\mathbb{R}^n.$ Recall by linear algebra, this is equivalent to checking whether the $n \times n$ matrix $A,$ whose $j$-th row is the row vector $\mathbf{v}_{i_{j+1}} - \mathbf{v}_{i_1},$ has nonzero determinant.

This is implemented in the code cell below. We also give an example to determine all candidate triangles for a square in $\mathbb{R}^2.$

In [49]:
def is_affinely_independent(l):
    l=np.array(l)
    A=(l-l[0])[1:]
    return np.linalg.det(A) !=0

def get_affinely_independent_vector_lists(V):
    n=len(V[0])
    all_lists=itertools.combinations(V,n+1)
    candidate_simplices=list(filter(lambda l: is_affinely_independent(l), all_lists))
    return candidate_simplices
    
V=[[0,0],[1,0],[0,1],[1,1]]
get_affinely_independent_vector_lists(V)

[([0, 0], [1, 0], [0, 1]),
 ([0, 0], [1, 0], [1, 1]),
 ([0, 0], [0, 1], [1, 1]),
 ([1, 0], [0, 1], [1, 1])]

# Determining Whether Candidate Simplices Have Disjoint Interiors

Given a candidate simplices $T$ with defining vertices $\mathbf{z}_1, \ \dots \ , \mathbf{z}_{n+1}$ and $U$ with defining vertices $\mathbf{w}_1, \ \dots \ , \mathbf{w}_{n+1},$ we determine whether $T$ and $U$ have disjoint interiors. Note $T$ and $U$ have joint interiors if and only if there exist $\alpha_1, \ \dots \ , \alpha_n, \beta_1, \ \dots \ , \beta_n \in \mathbb{R}^n$ such that each $\alpha_i, \beta_i >0$ and $\sum_{j=1}^n \alpha_j, \sum_{j=1}^n \beta_j <1,$ and 
$$\mathbf{z}_1 + \sum_{j=1}^n \alpha_j (\mathbf{z}_j - \mathbf{z}_1) =  \mathbf{w}_1 + \sum_{j=1}^n \beta_j (\mathbf{w}_j - \mathbf{w}_1).$$

We cannot analytically determine the coefficients $\alpha_i,\beta_i$ to solve the above equation. So our approach will be to take large, finite number of sampled coefficient sets of the form $\lbrace \alpha_1, \ \dots \ , \alpha_n \rbrace$ satisfying each $\alpha_i>0$  and $\sum_{j=1}^n \alpha_j < 1.$ For each such set, we compute the corresponding vector $\mathbf{p}=\mathbf{z}_1 + \sum_{j=1}^n \alpha_j (\mathbf{z}_j - \mathbf{z}_1).$ Using linear algebra, we solve the system $$\mathbf{p}=\mathbf{w}_1 + \sum_{j=1}^n \beta_j (\mathbf{w}_j - \mathbf{w}_1)$$ for $\beta_1, \ \dots \ , \beta_n.$ We determine if $\beta_i >0$ and $\sum_{j=1}^n \beta_j <1.$ If so, then $T$ and $U$ have joint interiors. However, if every aforementioned sampled coefficient set yields a noncompliant solution for $\beta_1, \ \dots \ , \beta_n,$ we declare $T$ and $U$ to have disjoint interiors.   


We implement this in the code cell below.

In [111]:
def random_points_in_interior(Z,n_trials=1000):
    n=len(Z[0])
    random_sampled_coefficients=[np.random.uniform(0,1,n) for i in range(n_trials)]
    valid_coefficients=np.array([alpha for alpha in random_sampled_coefficients if np.sum(alpha) < 1])
    points=[np.array(Z[0]) + np.sum([alpha[i]*(np.array(Z[i+1])-np.array(Z[0]))  for i in range(n)],axis=1) 
           for alpha in valid_coefficients]
    return points




def point_is_in_interior(W,p):
    p=np.array(p)
    W=np.array(W)
    A=((W-W[0])[1:]).T
    q=W[0]
    v= np.linalg.solve(A, p-q)
    for i in range(len(v)):
        if v[i] <= 0:
            return False
    if np.sum(v) >= 1:
        return False

    return True 
    
def pairwise_disjoint(Z,W,n_trials=10000):
    points=random_points_in_interior(Z,n_trials)
    try:
        next(p for p in points if point_is_in_interior(W,p))
        return False
    except:
        return True
    
    
Z=[[0, 0], [1, 0], [0, 1]]
W=[[1, 0], [0,0], [1, 1]]
pairwise_disjoint(Z,W)

False

In [ ]:
def